In [2]:
import numpy as np
import sympy as sp

$$F(x) = \sum^{u(x)}_{k=\alpha}{f(k)}$$

$$\Rightarrow v_{F(x)} = v_{f(x)}.T.P$$


- $T$: Triagle matrix
- $P$: Post matrix

### Create triagle matrix

In [28]:
class TriMatrix:
    def __init__(self):
        self.matrix = np.array([])
        self.post_matrix = np.array([])

    @staticmethod
    def _expand_poly(coeffs, degree):
        result = np.array([1], dtype=np.int64)
        for _ in range(degree):
            result = np.convolve(result, coeffs)
        return result

    def _poli_sum_on_row(self, base_number, row_index):
        sum = 0
        for index, value in enumerate(self.matrix[row_index]):
            sum += value*base_number**index
        return sum

    def call_triagle(self, level, below_num, upper_formula=np.array([0, 1])):

        # Call triagle matrix
        process_num = below_num - 1
        self.matrix = np.zeros((level, level+1))

        current_row_index = 0
        current_col_index = 1
        while True:
            if (current_row_index == 0) and (current_col_index == 1):
                self.matrix[current_row_index][current_col_index] = 1
                    
            elif current_col_index == 0:
                self.matrix[current_row_index][current_col_index] = - self._poli_sum_on_row(process_num, current_row_index)

            elif current_col_index == 1:
                self.matrix[current_row_index][current_col_index] = 1 - np.sum(self.matrix[current_row_index])

            else:
                self.matrix[current_row_index][current_col_index] = current_row_index/current_col_index * self.matrix[current_row_index-1][current_col_index-1]

            if (current_row_index == level-1) and (current_col_index == 0):
                break
            
            current_row_index += 1
            current_col_index += 1

            if current_row_index == level:
                current_row_index = level + 1 - current_col_index
                current_col_index = 0

        # Call post matrix to handle upper fence
        tmp_col_num = (len(upper_formula)-1)*(level+1)
        self.post_matrix = np.zeros((level+1, tmp_col_num))

        for degree in range(level+1):
            tmp_list = self._expand_poly(upper_formula, degree)
            for col_index in range(tmp_list.size):
                self.post_matrix[degree][col_index] = tmp_list[col_index]


        return self.matrix @ self.post_matrix



tri = TriMatrix()
tri.call_triagle(3, 2, [0, 1])
        

array([[-1.        ,  1.        ,  0.        ,  0.        ],
       [-1.        ,  0.5       ,  0.5       ,  0.        ],
       [-1.        ,  0.16666667,  0.5       ,  0.33333333]])

### Example 1: Find the sum formula:

$$f(x) = \sum^{x}_{x=3}{\sum^{x}_{x=2}{x+1}}$$

In [24]:
input_matrix = np.array([[1, 1]])

input_matrix @ tri.call_triagle(input_matrix.shape[1], 2) @ tri.call_triagle(input_matrix.shape[1]+1, 3)

array([[-3.        , -1.16666667,  1.        ,  0.16666667]])

$$\Rightarrow f(x) = -3 - \frac{7}{6}x + x^2 + \frac{1}{6}x^3$$

### Example 2: Find the sum formula 

$$f(x) = \sum^{x}_{x=3}{\sum^{2x+1}_{x=2}{(x^2+x+1)}}$$

In [39]:
input_matrix_2 = np.array([[1, 1, 1]])

input_matrix_2 @ tri.call_triagle(input_matrix_2.shape[1], 2, [1, 2]) @ tri.call_triagle(input_matrix_2.shape[1]+1, 3)

array([[-92.        ,   6.        ,   9.33333333,   4.        ,
          0.66666667]])

$$\Rightarrow f(x) = -92 + 6x + \frac{28}{3}x^2 + 4x^3 + \frac{2}{3}x^4 $$